In [67]:
from header import *

D = 10

model = nsp.model.SlRiemanGenerator(D, dtype=torch.float64)
set_seed(3342432)

X = np.random.randn(D**2, D**2)
X = (X + X.T)/2
mask = X>=(1/(D**2))
X[mask] = 0

loss = nsp.loss.MES_SL(torch.Tensor(X), [D,D], inv = model._inv)

model.reset_params()
loss(model.matrix()).backward()
slcg = RiemanSlCG(model, loss)
S, W = slcg._riemannian_grad(model._params)

solver_cg = UnitarySymmTs(RiemanSlCG, model, loss)
ret_cg = solver_cg.run(500, disable_message=False)

target loss      : 37.0190048218
initial loss     : 4189.9892410269




  0%|          | 0/500 [00:00<?, ?it/s]

stack in local minimum --> break loop


In [8]:
torch.matrix_exp(-1*S.data)

tensor([[ 0.5641, -0.0888],
        [ 0.6804,  1.6656]], dtype=torch.float64)

In [9]:
torch.linalg.norm(S)

tensor(1., dtype=torch.float64)

In [10]:
def objective(t):
    return loss(torch.matrix_exp(-t*S.data)@W.data).item()

In [11]:
torch.linalg.eigvals(S.data)

tensor([ 0.4748+0.j, -0.4748+0.j], dtype=torch.complex128)

In [14]:
loss(model.matrix())

tensor(11.8270, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [13]:
objective(1)

11.850399187478192

In [74]:
lr = slcg._golden(W.data, S.data)

_LinAlgError: torch.linalg.inv: The diagonal element 3 is zero, the inversion could not be completed because the input matrix is singular.

In [7]:
loss(model.matrix())

tensor(11.5067, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [8]:
loss(model.matrix()).backward()

In [10]:
model._params.grad

tensor([  1.2769, -12.3206,   7.5504,   7.5007,  -7.3384, -47.3078, -43.3932,
         28.8005,   1.4467,  27.3550,   8.5543, -16.6534, -12.1922, -75.3265,
        -72.0942,  45.8580], dtype=torch.float64)

In [38]:
model.matrix()

tensor([[ 1.2359, -0.7446,  0.8197,  1.7623],
        [-0.4031,  0.7149,  0.1083,  0.1054],
        [-0.3327, -0.3558, -0.2180,  0.1962],
        [-1.1024, -0.0299,  0.3727, -0.9463]], dtype=torch.float64,
       grad_fn=<ViewBackward0>)

In [25]:
S

tensor([[ 1.1503e+09,  1.0250e+09],
        [-1.2910e+09, -1.1503e+09]], dtype=torch.float64)